<a href="https://colab.research.google.com/github/guifav/emails_analysis/blob/main/Quando_os_maiores_portais_e_redes_sociais_do_mundo_enviam_emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# ANÁLISE PARA LINKEDIN: HORÁRIOS DOS GIGANTES
# "Quando os maiores portais e redes sociais do mundo enviam emails?"
# =============================================================================

from google.colab import drive
from datetime import datetime, timezone, timedelta
from collections import defaultdict, Counter
from email.utils import parsedate_to_datetime
import time
import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

drive.mount('/content/drive')

MBOX_FILE = "/content/drive/MyDrive/temp/Todos os e-mails, incluindo Spam e Lixeira-002.mbox"
BRT = timezone(timedelta(hours=-3))

# =============================================================================
# DEFINIÇÃO DOS GRUPOS
# =============================================================================

# Portais de notícias (domínio ou parte do from)
NEWS_PORTALS = {
    # Internacionais
    'nytimes': 'The New York Times',
    'new york times': 'The New York Times',
    'theinformation': 'The Information',
    'wsj.com': 'Wall Street Journal',
    'wall street journal': 'Wall Street Journal',
    'washingtonpost': 'Washington Post',
    'bloomberg': 'Bloomberg',
    'reuters': 'Reuters',
    'bbc': 'BBC',
    'cnn': 'CNN',
    'theguardian': 'The Guardian',
    'economist': 'The Economist',
    'ft.com': 'Financial Times',
    'financial times': 'Financial Times',
    'forbes': 'Forbes',
    'techcrunch': 'TechCrunch',
    'wired': 'Wired',
    'theverge': 'The Verge',
    'axios': 'Axios',
    'politico': 'Politico',
    'morning brew': 'Morning Brew',
    'morningbrew': 'Morning Brew',
    'theatlantic': 'The Atlantic',
    'newyorker': 'The New Yorker',
    'time.com': 'TIME',
    'huffpost': 'HuffPost',
    'businessinsider': 'Business Insider',
    'cnbc': 'CNBC',
    'fortune': 'Fortune',
    'fastcompany': 'Fast Company',
    'inc.com': 'Inc.',
    'venturebeat': 'VentureBeat',
    'mit technology': 'MIT Technology Review',
    'technologyreview': 'MIT Technology Review',
    'hbr.org': 'Harvard Business Review',
    'harvard business': 'Harvard Business Review',

    # Brasil
    'valor': 'Valor Econômico',
    'globo': 'O Globo / G1',
    'folha': 'Folha de S.Paulo',
    'estadao': 'Estadão',
    'uol': 'UOL',
    'exame': 'Exame',
    'infomoney': 'InfoMoney',
    'startse': 'StartSe',
    'neofeed': 'NeoFeed',
    'braziljournal': 'Brazil Journal',
    'meioemensagem': 'Meio & Mensagem',

    # Tech/Startup
    'startupgrind': 'Startup Grind',
    'startup grind': 'Startup Grind',
    'ycombinator': 'Y Combinator',
    'producthunt': 'Product Hunt',
    'techstars': 'Techstars',
    'sequoia': 'Sequoia Capital',
    'a]6z': 'Andreessen Horowitz',
    'a]6z.com': 'Andreessen Horowitz',
}

# Consultorias
CONSULTING = {
    'mckinsey': 'McKinsey & Company',
    'bain.com': 'Bain & Company',
    'bain ': 'Bain & Company',
    'bcg.com': 'Boston Consulting Group',
    'boston consulting': 'Boston Consulting Group',
    'ey.com': 'EY (Ernst & Young)',
    'ernst & young': 'EY (Ernst & Young)',
    'deloitte': 'Deloitte',
    'pwc': 'PwC',
    'kpmg': 'KPMG',
    'accenture': 'Accenture',
    'gartner': 'Gartner',
    'forrester': 'Forrester',
}

# Redes sociais
SOCIAL_NETWORKS = {
    'linkedin': 'LinkedIn',
    'youtube': 'YouTube',
    'facebook': 'Facebook',
    'facebookmail': 'Facebook',
    'meta.com': 'Meta',
    'twitter': 'Twitter/X',
    'x.com': 'Twitter/X',
    'instagram': 'Instagram',
    'snapchat': 'Snapchat',
    'tiktok': 'TikTok',
    'pinterest': 'Pinterest',
    'reddit': 'Reddit',
    'discord': 'Discord',
    'slack': 'Slack',
    'whatsapp': 'WhatsApp',
    'telegram': 'Telegram',
    'medium': 'Medium',
    'substack': 'Substack',
    'threads': 'Threads',
    'quora': 'Quora',
}

# Tech Giants
TECH_GIANTS = {
    'google': 'Google',
    'apple': 'Apple',
    'microsoft': 'Microsoft',
    'amazon': 'Amazon',
    'openai': 'OpenAI',
    'anthropic': 'Anthropic',
    'netflix': 'Netflix',
    'spotify': 'Spotify',
    'salesforce': 'Salesforce',
    'hubspot': 'HubSpot',
    'dropbox': 'Dropbox',
    'notion': 'Notion',
    'figma': 'Figma',
    'canva': 'Canva',
    'zoom': 'Zoom',
    'stripe': 'Stripe',
    'shopify': 'Shopify',
}

print("📂 Processando emails...")

# Estruturas de dados
all_senders = []  # (timestamp, from, category, source_name)
category_data = defaultdict(list)  # category -> list of timestamps
source_data = defaultdict(list)     # source_name -> list of timestamps

email_count = 0
matched_count = 0
start_time = time.time()

def categorize_sender(from_field):
    """Retorna (categoria, nome_fonte) ou (None, None)"""
    from_lower = from_field.lower()

    for key, name in NEWS_PORTALS.items():
        if key in from_lower:
            return ('Portais de Notícias', name)

    for key, name in CONSULTING.items():
        if key in from_lower:
            return ('Consultorias', name)

    for key, name in SOCIAL_NETWORKS.items():
        if key in from_lower:
            return ('Redes Sociais', name)

    for key, name in TECH_GIANTS.items():
        if key in from_lower:
            return ('Big Techs', name)

    return (None, None)

with open(MBOX_FILE, 'rb') as f:
    in_header = False
    current_date = None
    current_from = None

    for line in f:
        try:
            if line.startswith(b'From '):
                if current_date and current_from:
                    category, source_name = categorize_sender(current_from)
                    if category:
                        all_senders.append((current_date, current_from, category, source_name))
                        category_data[category].append(current_date)
                        source_data[source_name].append(current_date)
                        matched_count += 1

                in_header = True
                current_date = None
                current_from = None
                email_count += 1

                if email_count % 10000 == 0:
                    elapsed = time.time() - start_time
                    rate = email_count / elapsed if elapsed > 0 else 0
                    print(f"   {email_count:,} emails | {matched_count:,} matches ({rate:.0f}/s)")
                continue

            if line.strip() == b'':
                in_header = False
                continue

            if in_header:
                line_lower = line.lower()

                if line_lower.startswith(b'date:'):
                    date_str = line[5:].decode('utf-8', errors='ignore').strip()
                    try:
                        dt = parsedate_to_datetime(date_str)
                        current_date = dt.astimezone(BRT)
                    except:
                        pass

                elif line_lower.startswith(b'from:'):
                    current_from = line[5:].decode('utf-8', errors='ignore').strip()

        except:
            pass

# Último email
if current_date and current_from:
    category, source_name = categorize_sender(current_from)
    if category:
        all_senders.append((current_date, current_from, category, source_name))
        category_data[category].append(current_date)
        source_data[source_name].append(current_date)

elapsed = time.time() - start_time
print(f"\n✅ {elapsed/60:.1f} min | {email_count:,} emails | {matched_count:,} de grandes players")

DIAS_PT = {0: 'Seg', 1: 'Ter', 2: 'Qua', 3: 'Qui', 4: 'Sex', 5: 'Sáb', 6: 'Dom'}
DIAS_FULL = {0: 'Segunda', 1: 'Terça', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta', 5: 'Sábado', 6: 'Domingo'}

# =============================================================================
# CONFIGURAÇÃO VISUAL PARA LINKEDIN
# =============================================================================

plt.style.use('default')
plt.rcParams['figure.facecolor'] = '#0a0a0a'
plt.rcParams['axes.facecolor'] = '#0a0a0a'
plt.rcParams['axes.edgecolor'] = '#333333'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['text.color'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['grid.color'] = '#333333'
plt.rcParams['font.size'] = 12

COLORS = {
    'Portais de Notícias': '#FF6B6B',
    'Redes Sociais': '#4ECDC4',
    'Consultorias': '#FFE66D',
    'Big Techs': '#95E1D3',
}

# =============================================================================
# GRÁFICO 1: VISÃO GERAL POR CATEGORIA
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 14))
fig.suptitle('📊 Quando os Gigantes Enviam Emails?\nAnálise de 500.000+ emails',
             fontsize=20, fontweight='bold', color='white', y=1.02)

for idx, (category, timestamps_list) in enumerate(category_data.items()):
    ax = axes[idx // 2, idx % 2]

    hours = [ts.hour for ts in timestamps_list]
    hour_counts = Counter(hours)

    x = list(range(24))
    y = [hour_counts.get(h, 0) for h in x]

    color = COLORS.get(category, '#FFFFFF')
    bars = ax.bar(x, y, color=color, alpha=0.8, edgecolor='white', linewidth=0.5)

    # Destaca o horário de pico
    max_hour = max(hour_counts, key=hour_counts.get)
    bars[max_hour].set_color('#FFFFFF')
    bars[max_hour].set_alpha(1)

    ax.set_title(f'{category}\n({len(timestamps_list):,} emails)',
                 fontsize=14, fontweight='bold', color=color)
    ax.set_xlabel('Hora (Brasília)', fontsize=10)
    ax.set_ylabel('Quantidade', fontsize=10)
    ax.set_xticks(range(0, 24, 2))
    ax.set_xticklabels([f'{h}h' for h in range(0, 24, 2)])
    ax.grid(axis='y', alpha=0.3)

    # Anotação do pico
    ax.annotate(f'PICO: {max_hour}h', xy=(max_hour, hour_counts[max_hour]),
                xytext=(max_hour + 2, hour_counts[max_hour] * 1.1),
                fontsize=11, color='white', fontweight='bold',
                arrowprops=dict(arrowstyle='->', color='white', lw=1.5))

plt.tight_layout()
plt.savefig('grafico1_visao_geral.png', dpi=150, bbox_inches='tight',
            facecolor='#0a0a0a', edgecolor='none')
plt.show()
print("📁 Salvo: grafico1_visao_geral.png")

# =============================================================================
# GRÁFICO 2: COMPARATIVO HORA A HORA (TODAS CATEGORIAS)
# =============================================================================

fig, ax = plt.subplots(figsize=(16, 8))

x = np.arange(24)
width = 0.2
multiplier = 0

for category, timestamps_list in category_data.items():
    hours = [ts.hour for ts in timestamps_list]
    hour_counts = Counter(hours)
    total = len(timestamps_list)

    # Normaliza para percentual
    y = [(hour_counts.get(h, 0) / total * 100) for h in range(24)]

    offset = width * multiplier
    color = COLORS.get(category, '#FFFFFF')
    ax.bar(x + offset, y, width, label=category, color=color, alpha=0.85)
    multiplier += 1

ax.set_xlabel('Hora (Brasília)', fontsize=12)
ax.set_ylabel('% dos emails da categoria', fontsize=12)
ax.set_title('📈 Distribuição Horária: Portais vs Redes Sociais vs Consultorias vs Big Techs\n',
             fontsize=16, fontweight='bold')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels([f'{h}h' for h in range(24)])
ax.legend(loc='upper right', fontsize=11)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('grafico2_comparativo.png', dpi=150, bbox_inches='tight',
            facecolor='#0a0a0a', edgecolor='none')
plt.show()
print("📁 Salvo: grafico2_comparativo.png")

# =============================================================================
# GRÁFICO 3: TOP FONTES INDIVIDUAIS
# =============================================================================

# Pega top 15 fontes por volume
top_sources = sorted(source_data.items(), key=lambda x: len(x[1]), reverse=True)[:15]

fig, ax = plt.subplots(figsize=(14, 10))

sources_names = []
sources_hours = []
sources_counts = []
sources_colors = []

for source_name, timestamps_list in top_sources:
    hours = [ts.hour for ts in timestamps_list]
    most_common_hour = Counter(hours).most_common(1)[0][0]

    sources_names.append(source_name)
    sources_hours.append(most_common_hour)
    sources_counts.append(len(timestamps_list))

    # Determina cor pela categoria
    for ts, frm, cat, src in all_senders:
        if src == source_name:
            sources_colors.append(COLORS.get(cat, '#FFFFFF'))
            break

# Ordena por horário preferido
sorted_data = sorted(zip(sources_names, sources_hours, sources_counts, sources_colors),
                     key=lambda x: x[1])

y_pos = np.arange(len(sorted_data))
names = [d[0] for d in sorted_data]
hours = [d[1] for d in sorted_data]
counts = [d[2] for d in sorted_data]
colors = [d[3] for d in sorted_data]

bars = ax.barh(y_pos, hours, color=colors, alpha=0.85, edgecolor='white', linewidth=0.5)

ax.set_yticks(y_pos)
ax.set_yticklabels([f'{name} ({count:,})' for name, count in zip(names, counts)], fontsize=11)
ax.set_xlabel('Horário Preferido de Envio (Brasília)', fontsize=12)
ax.set_title('🎯 Horário Favorito dos Top 15 Remetentes\n(ordenado por hora)',
             fontsize=16, fontweight='bold')
ax.set_xlim(0, 24)
ax.set_xticks(range(0, 25, 2))
ax.set_xticklabels([f'{h}h' for h in range(0, 25, 2)])
ax.grid(axis='x', alpha=0.3)

# Adiciona hora no final da barra
for i, (bar, hour) in enumerate(zip(bars, hours)):
    ax.text(hour + 0.3, bar.get_y() + bar.get_height()/2, f'{hour}h',
            va='center', fontsize=10, fontweight='bold', color='white')

plt.tight_layout()
plt.savefig('grafico3_top_fontes.png', dpi=150, bbox_inches='tight',
            facecolor='#0a0a0a', edgecolor='none')
plt.show()
print("📁 Salvo: grafico3_top_fontes.png")

# =============================================================================
# GRÁFICO 4: HEATMAP HORA x DIA DA SEMANA (AGREGADO)
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(18, 14))

for idx, (category, timestamps_list) in enumerate(category_data.items()):
    ax = axes[idx // 2, idx % 2]

    # Matriz hora x dia
    matrix = np.zeros((7, 24))
    for ts in timestamps_list:
        matrix[ts.weekday()][ts.hour] += 1

    # Normaliza por linha (dia)
    row_sums = matrix.sum(axis=1, keepdims=True)
    matrix_pct = np.divide(matrix, row_sums, where=row_sums!=0) * 100

    color = COLORS.get(category, '#FFFFFF')
    cmap = plt.cm.colors.LinearSegmentedColormap.from_list('custom', ['#0a0a0a', color])

    im = ax.imshow(matrix_pct, cmap=cmap, aspect='auto')

    ax.set_xticks(range(0, 24, 2))
    ax.set_xticklabels([f'{h}h' for h in range(0, 24, 2)])
    ax.set_yticks(range(7))
    ax.set_yticklabels([DIAS_PT[d] for d in range(7)])
    ax.set_xlabel('Hora', fontsize=10)
    ax.set_title(f'{category}', fontsize=14, fontweight='bold', color=color)

    # Marca o pico de cada dia
    for day in range(7):
        max_hour = np.argmax(matrix_pct[day])
        if matrix_pct[day][max_hour] > 0:
            ax.plot(max_hour, day, 'w*', markersize=12)

fig.suptitle('🗓️ Heatmap: Quando Cada Categoria Envia por Dia da Semana\n(★ = pico do dia)',
             fontsize=18, fontweight='bold', y=1.02)

plt.tight_layout()
plt.savefig('grafico4_heatmap.png', dpi=150, bbox_inches='tight',
            facecolor='#0a0a0a', edgecolor='none')
plt.show()
print("📁 Salvo: grafico4_heatmap.png")

# =============================================================================
# GRÁFICO 5: REDES SOCIAIS - DETALHADO
# =============================================================================

social_sources = {k: v for k, v in source_data.items()
                  if any(k == SOCIAL_NETWORKS.get(key) for key in SOCIAL_NETWORKS)}

if social_sources:
    fig, ax = plt.subplots(figsize=(16, 8))

    x = np.arange(24)

    for source_name, timestamps_list in sorted(social_sources.items(),
                                                key=lambda x: len(x[1]), reverse=True)[:8]:
        hours = [ts.hour for ts in timestamps_list]
        hour_counts = Counter(hours)
        total = len(timestamps_list)
        y = [(hour_counts.get(h, 0) / total * 100) for h in range(24)]
        ax.plot(x, y, marker='o', linewidth=2, markersize=4,
                label=f'{source_name} ({total:,})', alpha=0.8)

    ax.set_xlabel('Hora (Brasília)', fontsize=12)
    ax.set_ylabel('% dos emails', fontsize=12)
    ax.set_title('📱 Redes Sociais: Quando Cada Uma Envia Notificações?\n',
                 fontsize=16, fontweight='bold')
    ax.set_xticks(range(24))
    ax.set_xticklabels([f'{h}h' for h in range(24)])
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig('grafico5_redes_sociais.png', dpi=150, bbox_inches='tight',
                facecolor='#0a0a0a', edgecolor='none')
    plt.show()
    print("📁 Salvo: grafico5_redes_sociais.png")

# =============================================================================
# GRÁFICO 6: PORTAIS DE NOTÍCIAS - DETALHADO
# =============================================================================

news_sources = {k: v for k, v in source_data.items()
                if any(k == NEWS_PORTALS.get(key) for key in NEWS_PORTALS)}

if news_sources:
    fig, ax = plt.subplots(figsize=(16, 8))

    x = np.arange(24)

    for source_name, timestamps_list in sorted(news_sources.items(),
                                                key=lambda x: len(x[1]), reverse=True)[:10]:
        hours = [ts.hour for ts in timestamps_list]
        hour_counts = Counter(hours)
        total = len(timestamps_list)
        y = [(hour_counts.get(h, 0) / total * 100) for h in range(24)]
        ax.plot(x, y, marker='o', linewidth=2, markersize=4,
                label=f'{source_name} ({total:,})', alpha=0.8)

    ax.set_xlabel('Hora (Brasília)', fontsize=12)
    ax.set_ylabel('% dos emails', fontsize=12)
    ax.set_title('📰 Portais de Notícias: Horário de Envio das Newsletters\n',
                 fontsize=16, fontweight='bold')
    ax.set_xticks(range(24))
    ax.set_xticklabels([f'{h}h' for h in range(24)])
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig('grafico6_portais_noticias.png', dpi=150, bbox_inches='tight',
                facecolor='#0a0a0a', edgecolor='none')
    plt.show()
    print("📁 Salvo: grafico6_portais_noticias.png")

# =============================================================================
# ANÁLISE TEXTUAL PARA LINKEDIN
# =============================================================================

print("\n" + "=" * 70)
print("📝 TEXTO PARA LINKEDIN")
print("=" * 70)

print("""
🔍 Analisei mais de 500.000 emails que recebi nos últimos anos para descobrir:

**Quando os gigantes da comunicação enviam seus emails?**

📊 PRINCIPAIS DESCOBERTAS:
""")

for category, timestamps_list in category_data.items():
    hours = [ts.hour for ts in timestamps_list]
    hour_counts = Counter(hours)
    top_hours = hour_counts.most_common(3)

    weekdays = [ts.weekday() for ts in timestamps_list]
    weekday_counts = Counter(weekdays)
    top_day = weekday_counts.most_common(1)[0]

    print(f"\n{'📰' if 'Notícias' in category else '📱' if 'Sociais' in category else '💼' if 'Consult' in category else '🖥️'} {category.upper()}")
    print(f"   • Horário favorito: {top_hours[0][0]}h ({top_hours[0][1]:,} emails)")
    print(f"   • Top 3 horários: {top_hours[0][0]}h, {top_hours[1][0]}h, {top_hours[2][0]}h")
    print(f"   • Dia preferido: {DIAS_FULL[top_day[0]]} ({top_day[1]:,} emails)")
    print(f"   • Total analisado: {len(timestamps_list):,} emails")

print("\n" + "-" * 50)
print("\n🎯 TOP 10 REMETENTES E SEUS HORÁRIOS FAVORITOS:\n")

for i, (source_name, timestamps_list) in enumerate(sorted(source_data.items(),
                                                           key=lambda x: len(x[1]),
                                                           reverse=True)[:10], 1):
    hours = [ts.hour for ts in timestamps_list]
    top_hour = Counter(hours).most_common(1)[0][0]
    print(f"   {i:2}. {source_name}: {top_hour}h ({len(timestamps_list):,} emails)")

print("\n" + "-" * 50)
print("\n💡 INSIGHTS:")

# Calcula horários mais vazios
all_timestamps = [ts for ts_list in category_data.values() for ts in ts_list]
all_hours = Counter([ts.hour for ts in all_timestamps])
empty_hours = sorted(all_hours.items(), key=lambda x: x[1])[:5]

# =============================================================================
# INSIGHTS DINÂMICOS (BASEADOS NOS DADOS REAIS)
# =============================================================================

print("\n💡 INSIGHTS (baseados nos dados):\n")

# Descobre horário de pico de cada categoria
for category, timestamps_list in category_data.items():
    hours = [ts.hour for ts in timestamps_list]
    hour_counts = Counter(hours)

    # Agrupa por período
    manha = sum(hour_counts.get(h, 0) for h in range(6, 12))      # 6h-11h
    tarde = sum(hour_counts.get(h, 0) for h in range(12, 18))     # 12h-17h
    noite = sum(hour_counts.get(h, 0) for h in range(18, 24))     # 18h-23h
    madrugada = sum(hour_counts.get(h, 0) for h in range(0, 6))   # 0h-5h
    total = len(timestamps_list)

    periodos = [
        ('manhã (6h-11h)', manha),
        ('tarde (12h-17h)', tarde),
        ('noite (18h-23h)', noite),
        ('madrugada (0h-5h)', madrugada)
    ]
    periodo_top = max(periodos, key=lambda x: x[1])
    pct = (periodo_top[1] / total * 100) if total > 0 else 0

    top_hour = hour_counts.most_common(1)[0][0]

    print(f"• {category}: concentra {pct:.0f}% dos envios na {periodo_top[0]}, pico às {top_hour}h")

# Horários com menor volume (oportunidade)
all_hours = Counter([ts.hour for ts_list in category_data.values() for ts in ts_list])
if all_hours:
    empty_hours = sorted(all_hours.items(), key=lambda x: x[1])[:5]
    print(f"\n• Horários com MENOR concorrência: {', '.join([f'{h}h' for h, c in empty_hours])}")

print("\n" + "=" * 70)
print("📁 GRÁFICOS SALVOS:")
print("   • grafico1_visao_geral.png")
print("   • grafico2_comparativo.png")
print("   • grafico3_top_fontes.png")
print("   • grafico4_heatmap.png")
print("   • grafico5_redes_sociais.png")
print("   • grafico6_portais_noticias.png")
print("=" * 70)

# Download automático
try:
    from google.colab import files
    for f in ['grafico1_visao_geral.png', 'grafico2_comparativo.png',
              'grafico3_top_fontes.png', 'grafico4_heatmap.png',
              'grafico5_redes_sociais.png', 'grafico6_portais_noticias.png']:
        files.download(f)
except:
    pass

print("\n✅ Análise completa!")